<a href="https://colab.research.google.com/github/cjfghk5697/multi-label-classification-Mnist/blob/main/Solve%20the%20Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/MyDrive/Test image2
!unzip -qq "/content/drive/MyDrive/multi-mnist zipfile/test_dirty_mnist_2nd.zip"

/content/drive/MyDrive/Test image2


In [2]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import numpy as np
import pandas as pd
import torch
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image


In [3]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(
                self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

In [4]:
#test Dataset 정의
transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

batch_size=32
sample_submission = pd.read_csv("/content/drive/MyDrive/sample_submission.csv")
testset = MnistDataset('/content/drive/MyDrive/Test image2', '/content/drive/MyDrive/sample_submission.csv', transforms_test)


In [11]:
from torchvision.models.efficientnet import efficientnet_b0
class MnistModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.model = efficientnet_b0(pretrained=True)
        self.classifier = nn.Linear(1000, 26)

    def forward(self, x):
        x = self.model(x)  
        x = self.classifier(x)

        return x

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = MnistModel().to(device)
path = "/content/drive/MyDrive/models/"
MODEL = "efficientnet5"
model_list=[]

model.load_state_dict(torch.load('/content/drive/MyDrive/models/0603 model.pth'), strict=False)

<All keys matched successfully>

In [13]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')
test_loader = DataLoader(testset, batch_size=32, num_workers=4)
model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    outputs = outputs > 0.5
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.long().squeeze(0).detach().cpu().numpy()
    
submit.to_csv('/content/drive/MyDrive/models/baseline_prediction3.csv', index=False)